# FIFA ETL
- - - -
### ETL using 2018 and 2019 FIFA player data

In [212]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [92]:
# Importing our FIFA data

fifa_2019_df = pd.read_csv("data.csv", low_memory = False)
fifa_2018_df = pd.read_csv("CompleteDataset.csv", low_memory = False)

fifa_2018_df.head(3)

,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0


In [93]:
# Picking out the data we want from each DF

clean_fifa_2019_df = fifa_2019_df[['ID', 'Name', 'Age', 'Nationality','Overall','Position','Club', 'Value', 'Wage']]
clean_fifa_2018_df = fifa_2018_df[['ID', 'Name', 'Age', 'Nationality','Overall','Club', 'Value', 'Wage']]
clean_fifa_2019_df.head(3)

,ID,Name,Age,Nationality,Overall,Position,Club,Value,Wage
0,158023,L. Messi,31,Argentina,94,RF,FC Barcelona,€110.5M,€565K
1,20801,Cristiano Ronaldo,33,Portugal,94,ST,Juventus,€77M,€405K
2,190871,Neymar Jr,26,Brazil,92,LW,Paris Saint-Germain,€118.5M,€290K


In [94]:
# MERGE ON ID

big_fifa_df = pd.merge(clean_fifa_2018_df,clean_fifa_2019_df,how = 'inner', on = 'ID',suffixes = (" 2018"," 2019"))
big_fifa_df.head(3)

,ID,Name 2018,Age 2018,Nationality 2018,Overall 2018,Club 2018,Value 2018,Wage 2018,Name 2019,Age 2019,Nationality 2019,Overall 2019,Position,Club 2019,Value 2019,Wage 2019
0,20801,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,€95.5M,€565K,Cristiano Ronaldo,33,Portugal,94,ST,Juventus,€77M,€405K
1,158023,L. Messi,30,Argentina,93,FC Barcelona,€105M,€565K,L. Messi,31,Argentina,94,RF,FC Barcelona,€110.5M,€565K
2,190871,Neymar,25,Brazil,92,Paris Saint-Germain,€123M,€280K,Neymar Jr,26,Brazil,92,LW,Paris Saint-Germain,€118.5M,€290K


In [95]:
# Selecting and reorganizing the columns after the merge

big_fifa_df_clean = big_fifa_df[['ID', 'Name 2018', 'Age 2019','Nationality 2019','Club 2018','Club 2019'
                                 , 'Overall 2018','Overall 2019', 'Value 2018','Value 2019'
                                 ,'Wage 2018','Wage 2019']]
big_fifa_df_clean.head(3)

,ID,Name 2018,Age 2019,Nationality 2019,Club 2018,Club 2019,Overall 2018,Overall 2019,Value 2018,Value 2019,Wage 2018,Wage 2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,€95.5M,€77M,€565K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,€105M,€110.5M,€565K,€565K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,€123M,€118.5M,€280K,€290K


In [96]:
# Renaming some of the columns to make SQL happy :)

big_fifa_df_clean_rename = big_fifa_df_clean.rename(columns={
    'Name 2018':'Name', 'Age 2019':'Age', 'Nationality 2019':'Nationality', 'Club 2018':'Club_2018',
       'Club 2019':'Club_2019', 'Overall 2018':'Overall_2018', 'Overall 2019':'Overall_2019', 'Value 2018':'Value_2018', 'Value 2019':'Value_2019',
       'Wage 2018':'Wage_2018', 'Wage 2019':'Wage_2019'
})

big_fifa_df_clean_rename.head(3)

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,€95.5M,€77M,€565K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,€105M,€110.5M,€565K,€565K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,€123M,€118.5M,€280K,€290K


In [124]:
# We noticed some of the players are duplicates, so....

soccer = big_fifa_df_clean_rename.drop_duplicates('ID')

In [125]:
soccer.head(2)

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,€95.5M,€77M,€565K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,€105M,€110.5M,€565K,€565K


In [126]:
# Converting the euros to dollars for the VALUE2018 and swaping it out in the table

value_18 = list(soccer['Value_2018'])
value_18_clean = []
gross_values = []
for value in value_18:
    
    if 'K' in value:
        make_float = float(value.replace('€','').replace('K',''))
        dollars = make_float*1.25
        f_dollars = '${:,.1f}K'.format(dollars)
        value_18_clean.append(f_dollars)
        
    elif 'M' in value:
        make_float = float(value.replace('€','').replace('M',''))
        dollars = make_float*1.25
        f_dollars = '${:,.1f}M'.format(dollars)
        value_18_clean.append(f_dollars)
        
    elif '€0' == value:
        value_18_clean.append('$0')
        
soccer['Value_2018'] = value_18_clean

C:\Users\Jacks\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [127]:
soccer.head()

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,$119.4M,€77M,€565K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,$131.2M,€110.5M,€565K,€565K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,$153.8M,€118.5M,€280K,€290K
3,176580,L. Suárez,31,Uruguay,FC Barcelona,FC Barcelona,92,91,$121.2M,€80M,€510K,€455K
4,167495,M. Neuer,32,Germany,FC Bayern Munich,FC Bayern München,92,89,$76.2M,€38M,€230K,€130K


In [128]:
# Converting the euros to dollars for the VALUE2019 and swaping it out in the table

value_19 = list(soccer['Value_2019'])
value_19_clean = []
gross_values = []
for value in value_19:
    
    if 'K' in value:
        make_float = float(value.replace('€','').replace('K',''))
        dollars = make_float*1.1
        f_dollars = '${:,.1f}K'.format(dollars)
        value_19_clean.append(f_dollars)
        
    elif 'M' in value:
        make_float = float(value.replace('€','').replace('M',''))
        dollars = make_float*1.1
        f_dollars = '${:,.1f}M'.format(dollars)
        value_19_clean.append(f_dollars)
        
    elif '€0' == value:
        value_19_clean.append('$0')
        
soccer['Value_2019'] = value_19_clean

C:\Users\Jacks\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [129]:
soccer.head()

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,$119.4M,$84.7M,€565K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,$131.2M,$121.6M,€565K,€565K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,$153.8M,$130.4M,€280K,€290K
3,176580,L. Suárez,31,Uruguay,FC Barcelona,FC Barcelona,92,91,$121.2M,$88.0M,€510K,€455K
4,167495,M. Neuer,32,Germany,FC Bayern Munich,FC Bayern München,92,89,$76.2M,$41.8M,€230K,€130K


In [136]:
# Converting the euros to dollars for the WAGE2018 and swaping it out in the table

wage_18 = list(soccer['Wage_2018'])
wage_18_clean = []
gross_values = []
for wage in wage_18:
    
    if 'K' in wage:
        make_float = float(wage.replace('€','').replace('K',''))
        dollars = make_float*1.25
        f_dollars = '${:,.1f}K'.format(dollars)
        wage_18_clean.append(f_dollars)
        
    elif '€0' == wage:
        wage_18_clean.append('$0')

        
soccer['Wage_2018'] = wage_18_clean

C:\Users\Jacks\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [137]:
soccer.head()

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,$119.4M,$84.7M,$706.2K,€405K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,$131.2M,$121.6M,$706.2K,€565K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,$153.8M,$130.4M,$350.0K,€290K
3,176580,L. Suárez,31,Uruguay,FC Barcelona,FC Barcelona,92,91,$121.2M,$88.0M,$637.5K,€455K
4,167495,M. Neuer,32,Germany,FC Bayern Munich,FC Bayern München,92,89,$76.2M,$41.8M,$287.5K,€130K


In [138]:
# Converting the euros to dollars for the WAGE2019 and swaping it out in the table

wage_19 = list(soccer['Wage_2019'])
wage_19_clean = []
gross_values = []
for wage in wage_19:
    
    if 'K' in wage:
        make_float = float(wage.replace('€','').replace('K',''))
        dollars = make_float*1.25
        f_dollars = '${:,.1f}K'.format(dollars)
        wage_19_clean.append(f_dollars)
        
    elif '€0' == wage:
        wage_19_clean.append('$0')

        
soccer['Wage_2019'] = wage_19_clean

C:\Users\Jacks\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [103]:
# Converting the euros to dollars for the WAGE2019 and swaping it out in the table

wage_19 = list(soccer['Wage_2019'])
wage_19_clean = []
gross_values = []
for wage in wage_19:
    
    if 'K' in wage:
        make_float = float(value.replace('€','').replace('K',''))
        dollars = make_float*1.25
        f_dollars = '${:,.1f}K'.format(dollars)
        wage_19_clean.append(f_dollars)
        
    elif 'M' in value:
        make_float = float(wage.replace('€','').replace('M',''))
        dollars = make_float*1.
        f_dollars = '${:,.1f}M'.format(dollars)
        wage_19_clean.append(f_dollars)
        
    elif '€0' == wage:
        wage_19_clean.append('$0')
        
soccer['Wage_2019'] = wage_19_clean

C:\Users\Jacks\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [217]:
soccer.head(3)

,ID,Name,Age,Nationality,Club_2018,Club_2019,Overall_2018,Overall_2019,Value_2018,Value_2019,Wage_2018,Wage_2019
0,20801,Cristiano Ronaldo,33,Portugal,Real Madrid CF,Juventus,94,94,$119.4M,$84.7M,$706.2K,$506.2K
1,158023,L. Messi,31,Argentina,FC Barcelona,FC Barcelona,93,94,$131.2M,$121.6M,$706.2K,$706.2K
2,190871,Neymar,26,Brazil,Paris Saint-Germain,Paris Saint-Germain,92,92,$153.8M,$130.4M,$350.0K,$362.5K


In [198]:
# make a DF of all the shooting stats

attack_stats_df = fifa_2018_df[['Name', 'Age', 'Nationality', 'Overall', 'Club', 'Curve', 'Finishing',
       'Free kick accuracy', 'Heading accuracy', 'Long shots', 'Penalties', 'Shot power','Volleys','Preferred Positions']]
attack_stats_df.head(3)

,Name,Age,Nationality,Overall,Club,Curve,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,Volleys,Preferred Positions
0,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,81,94,76,88,92,85,94,88,ST LW
1,L. Messi,30,Argentina,93,FC Barcelona,89,95,90,71,88,74,85,85,RW
2,Neymar,25,Brazil,92,Paris Saint-Germain,81,89,84,62,77,81,80,83,LW


In [200]:
# Get only the attacking players and their shooting stats

def my_rule(row):
    positions = ['RF', 'ST','LW','LF','RS','CAM','LS','LAM','RW','RAM','CF']
    items = row.split(' ')
    for item in items:
        if item in positions:
            return True
    return False

only_attacking_players = attack_stats_df[attack_stats_df['Preferred Positions'].apply(lambda row: my_rule(row))]

only_attacking_players.head(3)

,Name,Age,Nationality,Overall,Club,Curve,Finishing,Free kick accuracy,Heading accuracy,Long shots,Penalties,Shot power,Volleys,Preferred Positions
0,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,81,94,76,88,92,85,94,88,ST LW
1,L. Messi,30,Argentina,93,FC Barcelona,89,95,90,71,88,74,85,85,RW
2,Neymar,25,Brazil,92,Paris Saint-Germain,81,89,84,62,77,81,80,83,LW


In [210]:
# Drop preferred positions column, reorder all the columns, and rename all the columns

# Drop preferred positions
df1 = only_attacking_players[['Name', 'Age', 'Nationality', 'Overall', 'Club', 'Curve', 'Finishing',
       'Free kick accuracy', 'Heading accuracy', 'Long shots', 'Penalties',
       'Shot power', 'Volleys']]

# Reorder the columns
df2 = df1[['Name', 'Age', 'Nationality', 'Overall', 'Club', 'Finishing','Long shots','Shot power', 'Free kick accuracy','Volleys','Penalties', 'Heading accuracy','Curve']]

# Rename the columns
atk_players = df2.rename(columns={'Long shots':'Long_shots','Shot power':'Shot_power',
                                  'Free kick accuracy':'Free_kick_accuracy',
                                  'Heading accuracy':'Heading_accuracy'})
atk_players.head(3)

,Name,Age,Nationality,Overall,Club,Finishing,Long_shots,Shot_power,Free_kick_accuracy,Volleys,Penalties,Heading_accuracy,Curve
0,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,94,92,94,76,88,85,88,81
1,L. Messi,30,Argentina,93,FC Barcelona,95,88,85,90,85,74,71,89
2,Neymar,25,Brazil,92,Paris Saint-Germain,89,77,80,84,83,81,62,81


In [214]:
# Put it into SQL

# establish connection
rds_connection_string = "postgres:12323abcd@localhost:5432/fifa_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [215]:
# put the money table into the DB, MAKE SURE TO CREATE THE DB FIRST

soccer.to_sql(name='fifa_money', con=engine, if_exists='append', index=False)


In [216]:
# put the attacking stats table in the DB

atk_players.to_sql(name='attacking_player_stats', con=engine, if_exists='append', index=False)